In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv(os.path.join('train.csv'))
val = pd.read_csv(os.path.join('validation.csv'))
test = pd.read_csv(os.path.join('test.csv'))

In [3]:
val.columns

Index(['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [4]:
# def create_indiv_data(data):
#     advertisers = sorted(set(data['advertiser'].values))
#     indiv_data = []
#     for advertiser in advertisers:
#         indiv_data.append(data[data['advertiser'] == advertiser])
#     return indiv_data

In [5]:
def compare_performance(metrics, best_metrics):
    return metrics[0] >= best_metrics[0] and metrics[1] >= best_metrics[1]

In [6]:
val_pred = val.copy()

In [7]:
click_list = []
CTR_list = []
cost_list = []
results = []
bidprce_list = list(range(301))

for bidprice in bidprce_list:
    print('processing ' + str(bidprice))
    val_pred['bidprice'] = bidprice
    budget = 6250
    suc_bids = val_pred.query('bidprice >= payprice and bidprice >= slotprice')
    cost = 0
    clicks = 0
    imps = 0
    for index, row in suc_bids.iterrows():
        if cost <= budget:
            cost += row['payprice'] / 1000
            clicks += row['click']
            imps += 1
    CTR = clicks / imps * 100
    CTR_list.append(CTR * 1000)
    click_list.append(clicks)
    cost_list.append(cost / 100)

    eCPC = cost / clicks if clicks > 0 else float('inf')
    metrics = [clicks, CTR, cost, cost / imps * 1000, eCPC]
    results.append([bidprice, clicks, CTR, cost, cost / imps * 1000, eCPC])
    
results = pd.DataFrame(results)
results = results.rename(columns={0: "bidprice", 1: "clicks",
    2: "CTR", 3: "cost", 4: "avg CPM", 5: "eCPC"})
results.to_csv(os.path.join('constant_bid.csv'), index=False)

processing 0
processing 1
processing 2
processing 3
processing 4
processing 5
processing 6
processing 7
processing 8
processing 9
processing 10
processing 11
processing 12
processing 13
processing 14
processing 15
processing 16
processing 17
processing 18
processing 19
processing 20
processing 21
processing 22
processing 23
processing 24
processing 25
processing 26
processing 27
processing 28
processing 29
processing 30
processing 31
processing 32
processing 33
processing 34
processing 35
processing 36
processing 37
processing 38
processing 39
processing 40
processing 41
processing 42
processing 43
processing 44
processing 45
processing 46
processing 47
processing 48
processing 49
processing 50
processing 51
processing 52
processing 53
processing 54
processing 55
processing 56
processing 57
processing 58
processing 59
processing 60
processing 61
processing 62
processing 63
processing 64
processing 65
processing 66
processing 67
processing 68
processing 69
processing 70
processing 71
pr

In [15]:
results.loc[results.clicks == max(results['clicks'])]

,bidprice,clicks,CTR,cost,avg CPM,eCPC
77,77,68,0.046301,6250.036,42.556044,91.912294
79,79,68,0.046602,6250.018,42.832399,91.912029


In [ ]:
ys = [click_list, CTR_list, cost_list]
labels = ['clicks', 'CTR', 'cost']

for y, label in zip(ys, labels):
    plt.plot(bidprce_list, y, label=label)
plt.legend()
plt.xlabel('bidprice')
plt.ylabel('Clicks, cost and CTR')
plt.title('Investigating the impacts of changing bidprice on the performance')
plt.show()

In [ ]:
val_pred.head()